In [1]:
import re
from collections import Counter
import itertools
from sklearn.feature_extraction.text import CountVectorizer
import re

In [2]:
documents = "Xin chào tất cả mọi ngưoời. Hôm nay chúng ta sẽ học về xử lý ngôn ngữ tự nhiên"

# Thay đổi vector biểu diễn dưới dạng

```
{'contains(waste)':False, 'contains(lot)': False, ... }
```

```
{'waste':10, 'lot': 5, ...} Từ - Tấn số
```

In [3]:
def doc_words_from_document(doc):
    """
    Trích danh sách từ (lowercase) từ một document. doc có thể là (list_tu, nhãn), hoặc list_tu, hoặc string.
    """
    # Nếu là tuple (words, label)
    if isinstance(doc, tuple) and len(doc) >= 1:
        doc = doc[0]
    if isinstance(doc, list):
        words = [str(w).lower() for w in doc]
    elif isinstance(doc, str):
        # Tokenize đơn giản cho tiếng Việt/Anh (giữ chữ/số)
        words = re.findall(r"\w+", doc.lower())
    else:
        words = []
    return words

In [4]:
# Boolean contains(word) và đếm tần suất
tokens = doc_words_from_document(documents)

In [5]:
# Count features: {'waste': 10, 'lot': 5, ...}
# Chỉ tạo đặc trưng đếm tại đây; đặc trưng boolean sẽ tạo sau khi xác định vocab
features_counts = dict(Counter(tokens))

In [6]:
# Vocabulary và đặc trưng boolean
vocab = sorted(set(tokens))
features_contains = {f"contains({w})": (w in set(tokens)) for w in vocab}

In [7]:
print(f"Số từ trong tài liệu mẫu: {len(tokens)}")
print(f"Kích thước vocab: {len(vocab)}")
print(f"Số đặc trưng dạng boolean: {len(features_contains)}")
print(f"Số đặc trưng dạng đếm: {len(features_counts)}")

Số từ trong tài liệu mẫu: 19
Kích thước vocab: 19
Số đặc trưng dạng boolean: 19
Số đặc trưng dạng đếm: 19


In [8]:
# Hiển thị một vài phần tử mẫu
print("Một vài đặc trưng boolean:")
for k, v in itertools.islice(features_contains.items(), 5):
    print(f"\"{k}\": {v}")

print("\nMột vài đặc trưng đếm:")
for k, v in itertools.islice(features_counts.items(), 5):
    print(f"\"{k}\": {v}")

Một vài đặc trưng boolean:
"contains(chào)": True
"contains(chúng)": True
"contains(cả)": True
"contains(hôm)": True
"contains(học)": True

Một vài đặc trưng đếm:
"xin": 1
"chào": 1
"tất": 1
"cả": 1
"mọi": 1


# Xây dựng hạng biểu diễn encode dạng vector trong sklearn

```
Vocab{waste, lot, many} --> (?, ?, ?)
```

In [9]:
# 2Encode dạng vector trong sklearn (CountVectorizer)
# Hàm chuyển document về chuỗi (nếu đầu vào là tokens)
def doc_to_text(doc):
    words = doc_words_from_document(doc)
    return " ".join(words)

# Xây dựng corpus văn bản
if isinstance(documents, list):
    corpus_texts = [doc_to_text(d) for d in documents[:200]]  # dùng 200 tài liệu đầu
elif isinstance(documents, str):
    corpus_texts = [doc_to_text(documents)]
else:
    corpus_texts = [
        "Khi bão đổ bộ sóng gió nổi lên",
        "Khi bão đổ bộ xóng gió nổi lên",
        "Bão lớn gây mưa to gió mạnh"
    ]


# Xây dựng vocabulary
# Lấy vocab từ chính corpus
tokens_all = []
for t in corpus_texts:
    tokens_all.extend(re.findall(r"\w+", t.lower()))
vocab_for_vectorizer = sorted(set(tokens_all))

vectorizer = CountVectorizer(vocabulary=vocab_for_vectorizer, lowercase=True, token_pattern=r"(?u)\b\w+\b")
X = vectorizer.transform(corpus_texts)

In [10]:
print("Kích thước vocab:", len(vectorizer.vocabulary_))
print("Kích thước ma trận (số văn bản, số đặc trưng):", X.shape)

Kích thước vocab: 19
Kích thước ma trận (số văn bản, số đặc trưng): (1, 19)


In [11]:
# Xem vector của văn bản đầu tiên
first_vec = X[0].toarray()[0]

# Lấy các đặc trưng khác 0 để quan sát
inv_vocab = {idx: term for term, idx in vectorizer.vocabulary_.items()}
non_zero = [(inv_vocab[i], int(c)) for i, c in enumerate(first_vec) if c > 0]
print("Đặc trưng khác 0 của văn bản [0]:", non_zero[:20])

Đặc trưng khác 0 của văn bản [0]: [('chào', 1), ('chúng', 1), ('cả', 1), ('hôm', 1), ('học', 1), ('lý', 1), ('mọi', 1), ('nay', 1), ('ngôn', 1), ('ngưoời', 1), ('ngữ', 1), ('nhiên', 1), ('sẽ', 1), ('ta', 1), ('tất', 1), ('tự', 1), ('về', 1), ('xin', 1), ('xử', 1)]
